Have changed the name of layer? Use it!

In [17]:
from pathlib import Path
import torch

from cadft.utils import (
    add_args,
    gen_keys_l,
    gen_model_dict,
    load_model,
)

key_map = {
    "fc4.weight": "fcout1.weight",
    "fc4.bias": "fcout1.bias",
    "fc5.weight": "fcout2.weight",
    "fc5.bias": "fcout2.bias",
}

ATOM_LIST = [
    "H",
    "C",
]

keys_l = gen_keys_l(ATOM_LIST)
device = torch.device("cpu")
model_dict = gen_model_dict(keys_l, 800, device)

dir_load = Path(f"checkpoints/checkpoint-ccdft-2024-05-18-19-14-05-{800}/")
for key in keys_l:
    for i_str in ["1", "2"]:
        key_i_str = key + i_str
        list_of_path = list(dir_load.glob(f"{key}-{i_str}*.pth"))
        if len(list_of_path) == 0:
            print(f"No model found for {key_i_str}, use random initialization.")
            continue
        load_path = max(list_of_path, key=lambda p: p.stat().st_ctime)
        state_dict = torch.load(load_path, map_location=device)
        renamed_dict = {key_map.get(k, k): v for k, v in state_dict.items()}
        model_dict[key_i_str].load_state_dict(renamed_dict)
        torch.save(
            renamed_dict,
            dir_load / f"{key}-{i_str}-{0}.pth",
        )